# Visualize Decoded Results

In [1]:
from nuscenes.nuscenes import NuScenes
from pyquaternion.quaternion import Quaternion
from nuscenes.utils.geometry_utils import view_points, box_in_image, BoxVisibility, transform_matrix
from nuscenes.utils.data_classes import PointCloud, LidarPointCloud, RadarPointCloud, Box
import numpy as np
import shutil
from PIL import Image
import os.path as osp
import cv2
import matplotlib.pyplot as plt
import open3d as o3d
from matplotlib import cm
from sphere import Sphere
from tqdm.auto import tqdm, trange
from tqdm.contrib.concurrent import process_map, thread_map
from functools import partial
from dh_grid import DHGrid
from sphere import Sphere
from img_sphere import ImageSphere
from metrics import *
from average_meter import AverageMeter

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [5]:
# export_ds = '/mnt/data/datasets/nuscenes/processed'
export_ds = '/media/scratch/berlukas/nuscenes'

decoded_filename = f"{export_ds}/sem_clouds_decoded.npy"
cloud_filename = f"{export_ds}/sem_clouds.npy"
decoded_features = np.load(decoded_filename)
cloud_features = np.load(cloud_filename)
print(f"Shape of the decoded signal: {decoded_features.shape}.")
print(f"Shape of the input signal: {cloud_features.shape}.")

Shape of the decoded signal: (400, 9, 200, 200).
Shape of the input signal: (11230, 3, 200, 200).


In [7]:
def CreateGrid(bw):
    n_grid = 2 * bw
    k = 0;
    points = np.empty([2, n_grid, n_grid])
    for i in range(n_grid):
        for j in range(n_grid):
            points[0, i, j] = (np.pi*(2*i+1))/(4*bw)
            points[1, i, j] = (2*np.pi*j)/(2*bw);
            k = k + 1;
    return points

def createGrid_old(bw):
        n_grid = 2 * bw
        k = 0;
        points = np.empty([n_grid * n_grid, 2])
        for i in range(n_grid):
            for j in range(n_grid):
                points[k, 0] = (np.pi*(2*i+1))/(4*bw)
                points[k, 1] = (2*np.pi*j)/(2*bw);
                k = k + 1;
        return points
    
def convertGridToEuclidean_old(grid):
    cart_grid = np.zeros([ grid.shape[0], 3])
    cart_grid[:,0] = np.multiply(np.sin(grid[:,0]), np.cos(grid[:,1]))
    cart_grid[:,1] = np.multiply(np.sin(grid[:,0]), np.sin(grid[:,1]))
    cart_grid[:,2] = np.cos(grid[:,0])    
    return cart_grid

def mapIntensityToRGB(i):
    return cm.jet(plt.Normalize(min(i), max(i))(i))

class SamplingPointCloud(PointCloud):

    @staticmethod
    def nbr_dims() -> int:
        return 4
    
    @classmethod
    def from_file(cls, file_name: str) -> 'SamplingPointCloud':
        return None
    
    @classmethod
    def from_bw(cls, bw, scale = 100) -> 'SamplingPointCloud':
        grid = createGrid_old(bw)
        xyz_grid = convertGridToEuclidean_old(grid) * scale
        intensities = np.zeros((xyz_grid.shape[0],1))
        sampling_grid = np.hstack((xyz_grid, np.ones((xyz_grid.shape[0], 1), dtype=xyz_grid.dtype)))
        return cls(sampling_grid.T)

def create_spherical_pc(feature, trans = 0, bw = 100):
    pc = SamplingPointCloud.from_bw(bw, 1)   
    points_xyz = pc.points.T[:,0:3]
    points_xyz[:,0] = points_xyz[:,0] + trans
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points_xyz[:, 0:3])
    colors = mapIntensityToRGB(feature[:, 0])
    pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])
    return pcd    

def create_cloud_pc(cloud, trans = 0):
    cloud[:,0] = cloud[:,0] + trans
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(cloud[:, 0:3])
    if cloud.shape[1] == 4:
        colors = mapIntensityToRGB(cloud[:, 3])
        pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])
    return pcd

def compare_estimation_sphere(decoded, gt, bw = 100):  
    decoded_pc = create_spherical_pc(decoded, trans=0, bw=bw)
    gt_pc = create_spherical_pc(gt, trans=2.5, bw=bw)
    o3d.visualization.draw_geometries([decoded_pc, gt_pc])    
    
def compare_estimation_clouds(decoded, gt, bw = 100):  
    decoded_pc = create_cloud_pc(decoded, trans=0)
    gt_pc = create_cloud_pc(gt, trans=100)
    o3d.visualization.draw_geometries([decoded_pc, gt_pc])
    
def backproject_cloud(spherical, distance, bw = 100):    
    grid, _ = DHGrid.CreateGrid(bw)
    n_points = grid.shape[1] * grid.shape[2]
    cart_sphere = np.zeros([n_points, 4])
    k = 0
    for i in range(0, grid.shape[1]):
        for j in range(0, grid.shape[2]):
            dist = distance[i,j]
            if dist <= 0:
                continue
            cart_sphere[k,0] = dist * np.multiply(np.cos(grid[1,i,j]), np.sin(grid[0,i,j]))
            cart_sphere[k,1] = dist * np.multiply(np.sin(grid[1,i,j]), np.sin(grid[0,i,j]))
            cart_sphere[k,2] = dist * np.cos(grid[0,i,j])
            cart_sphere[k,3] = spherical[i,j]
            k = k + 1
    return cart_sphere  


i = 0

cur_decoded = decoded_features[i, :, :, :]
cur_decoded = np.argmax(cur_decoded, axis=0)

cur_cloud = cloud_features[i, 2, :, :]
compare_estimation_sphere(cur_decoded, cur_cloud)

# est_cloud = backproject_cloud(cur_decoded, cur_input[0,:,:], bw)
# gt_cloud = backproject_cloud(cur_sem_sphere, cur_input[0,:,:], bw)

RuntimeError: [1;31m[Open3D ERROR] GLFW Error: X11: The DISPLAY environment variable is missing[0;m

In [ ]:
compare_estimation_sphere(cur_decoded, cur_cloud)
